In [ ]:
import sys
sys.path.append("./notebooks")
sys.path.append("./src")

In [ ]:
import os
import json
import requests
import geojson
import io
import math

import numpy as np
import matplotlib.pyplot as plt
import sentinelhub as seh

from datetime import datetime, timedelta, timezone
from shapely.geometry import shape
from shapely.ops import transform
from pyproj import Transformer
from dotenv import load_dotenv
from pathlib import Path
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from PIL import Image
from ipyleaflet import GeoJSON, Map, basemaps
from rasterio.io import MemoryFile

from utils.paths import get_data_path

from utils.sentinelhub_api import build_json_request

In [ ]:
start_date = "2020-06-01"
end_date = "2020-08-31"
collection_id = "sentinel-2-l2a"

geojson_path = get_data_path("blackForestPoly.geojson")#"blackForestPoly.geojson"

load_dotenv()
client_id = os.getenv('SENTINELHUB_CLIENT_ID')
client_secret = os.getenv('SENTINELHUB_CLIENT_SECRET')
token_url = os.getenv("SENTINELHUB_TOKEN_URL", "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token")

In [ ]:
#get geometry from file
with open(geojson_path) as f:
    geo_file = geojson.load(f)
    
geometry = geo_file['features'][0]['geometry']

In [ ]:
#get bbox to resize request plot
geom = shape(geometry)
project = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True).transform
geom_m = transform(project, geom)

# Get size in meters
minx, miny, maxx, maxy = geom_m.bounds
width_m = maxx - minx
height_m = maxy - miny

# Initial pixel dimensions at target resolution
width_px = width_m / 20
height_px = height_m / 20

# Scale down if larger than max_dim
scale = max(width_px / 2500, height_px / 2500, 1)
width_px = int(math.ceil(width_px / scale))
height_px = int(math.ceil(height_px / scale))

In [ ]:
import rasterio
from rasterio.transform import from_origin
from rasterio.windows import Window
from rasterio.windows import bounds

# Example grid: 1000x1000 pixels, 20 m resolution
_, _, height, width = res.shape
xmin, ymin, xmax, ymax = minx, miny, maxx, maxy
pixel_size = 20

# Build transform (top-left origin)
transform = from_origin(xmin, ymax, pixel_size, pixel_size)

# Example: slice indices (row_start:row_stop, col_start:col_stop)
row_start, row_stop = 4400, 5000
col_start, col_stop = 2250, 3350

# Create a rasterio Window for that slice
window = Window.from_slices((row_start, row_stop), (col_start, col_stop))

# Get bbox in map coordinates
bbox = bounds(window, transform=transform)
print(bbox)

In [ ]:
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

token = oauth.fetch_token(
    token_url=token_url,
    client_secret=client_secret,
    include_client_id=True
)

In [ ]:
json_request = build_json_request(width_px, height_px, datetime(2025, 6, 1), datetime(2025, 6, 30), "RGB", geometry=geometry, crs="CRS84")

In [ ]:
bbox = [882101.9766596204, 6172801.910631929, 904101.9766596204, 6184801.910631929]

width_x = (bbox[2] - bbox[0])/20
width_y = (bbox[3] - bbox[1])/20

json_request = build_json_request(width_x, width_y, datetime(2025, 6, 1), datetime(2025, 6, 30), "RGB", bbox=bbox)

In [ ]:
# Set the request URL and headers
url_request = "https://sh.dataspace.copernicus.eu/api/v1/process"
headers_request = {
    "Authorization": f"Bearer {token['access_token']}"
}

# Send the request
response = oauth.post(url_request, headers=headers_request, json=json_request)

In [ ]:
# Check if request succeeded
if response.status_code == 200:
    # read the image as numpy array
    image_arr = np.array(Image.open(io.BytesIO(response.content)))

    # plot the image for visualization
    plt.figure(figsize=(16,16))
    plt.axis('off')
    plt.tight_layout()
    plt.imshow(image_arr)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [ ]:
# Read image bytes into numpy array
with MemoryFile(response.content) as memfile:
    with memfile.open() as dataset:
        image = dataset.read()

In [ ]:
res = np.load("data2025.npy")

In [ ]:
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
cmap = plt.cm.Greens

# Map normalized NDVI values to RGBA colors
norm = Normalize(vmin=-1, vmax=1)

rgba_img = cmap(norm(res[0, 8, 4500:4505, 3345:3350]))
# Display the image
fig, ax = plt.subplots()
fig.set_size_inches(15, 15)
im = ax.imshow(rgba_img)
cbar = fig.colorbar(ScalarMappable(norm=norm, cmap=cmap), ax=ax, label='NDVI')

ax.axis('off')
plt.show()


In [ ]:
res[0, :, 4504:4505, 3346:3347]

In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

ndvi = res[0, 8, 4400:5000, 2250:3350]

# Define spatial info for your array
xmin, ymin, xmax, ymax = (882101.9766596204, 6172801.910631929, 904101.9766596204, 6184801.910631929)  # bounds in your CRS
pixel_size = 20  # 20 m per pixel
transform = from_origin(xmin, ymax, pixel_size, pixel_size)  # top-left origin

# Save as GeoTIFF
with rasterio.open(
    "ndvi.tif",
    "w",
    driver="GTiff",
    height=ndvi.shape[0],
    width=ndvi.shape[1],
    count=1,
    dtype=ndvi.dtype,
    crs="EPSG:3857",  # adjust to your CRS
    transform=transform,
) as dst:
    dst.write(ndvi, 1)